# Text and Sequence Generation with Hyena-GLT

This notebook demonstrates various text and sequence generation techniques using the Hyena-GLT framework. We'll explore different sampling strategies, conditioning approaches, and generation quality metrics.

## Setup and Imports

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Optional, Tuple
import logging
from tqdm import tqdm

from hyena_glt.models import HyenaGLT
from hyena_glt.tokenizers import GenomicTokenizer
from hyena_glt.utils import (
    plot_attention_maps,
    analyze_model_predictions,
    validate_sequence
)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Model and Tokenizer Setup

In [ ]:
# Initialize tokenizer
tokenizer = GenomicTokenizer(
    vocab_size=8192,
    special_tokens=['<PAD>', '<UNK>', '<BOS>', '<EOS>', '<MASK>']
)

# Model configuration
model_config = {
    'vocab_size': tokenizer.vocab_size,
    'hidden_dim': 512,
    'num_layers': 12,
    'max_seq_len': 2048,
    'hyena_order': 3,
    'dropout': 0.1
}

# Initialize model
model = HyenaGLT(**model_config)
model = model.to(device)
model.eval()

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Tokenizer vocabulary size: {tokenizer.vocab_size}")

## Generation Strategies

### 1. Greedy Decoding

In [ ]:
def greedy_generate(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    prompt: str,
    max_length: int = 100,
    device: torch.device = torch.device('cpu')
) -> str:
    """
    Generate text using greedy decoding (always pick most likely next token).
    """
    model.eval()
    
    # Tokenize prompt
    input_ids = tokenizer.encode(prompt)
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    
    generated_ids = input_ids.copy()
    
    with torch.no_grad():
        for _ in range(max_length):
            # Get model predictions
            outputs = model(input_tensor)
            logits = outputs['logits']  # [batch_size, seq_len, vocab_size]
            
            # Get next token (greedy)
            next_token_logits = logits[0, -1, :]  # Last position
            next_token_id = torch.argmax(next_token_logits).item()
            
            # Stop if EOS token
            if next_token_id == tokenizer.eos_token_id:
                break
            
            # Add to sequence
            generated_ids.append(next_token_id)
            
            # Update input for next iteration
            input_tensor = torch.cat([
                input_tensor,
                torch.tensor([[next_token_id]], dtype=torch.long).to(device)
            ], dim=1)
    
    return tokenizer.decode(generated_ids)

# Test greedy generation
prompt = "ATGCGATCGAT"
generated_text = greedy_generate(model, tokenizer, prompt, max_length=50, device=device)
print(f"Prompt: {prompt}")
print(f"Generated: {generated_text}")

### 2. Nucleus (Top-p) Sampling

In [ ]:
def top_p_sample(logits: torch.Tensor, p: float = 0.9) -> int:
    """
    Sample from top-p (nucleus) distribution.
    """
    # Sort logits in descending order
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    
    # Compute cumulative probabilities
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
    
    # Find cutoff index
    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1].clone()
    sorted_indices_to_remove[0] = False
    
    # Set logits to -inf for tokens outside nucleus
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = float('-inf')
    
    # Sample from filtered distribution
    probs = F.softmax(logits, dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    
    return next_token.item()

def nucleus_generate(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    prompt: str,
    max_length: int = 100,
    p: float = 0.9,
    temperature: float = 1.0,
    device: torch.device = torch.device('cpu')
) -> str:
    """
    Generate text using nucleus (top-p) sampling.
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt)
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    
    generated_ids = input_ids.copy()
    
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(input_tensor)
            logits = outputs['logits'][0, -1, :] / temperature
            
            next_token_id = top_p_sample(logits, p)
            
            if next_token_id == tokenizer.eos_token_id:
                break
                
            generated_ids.append(next_token_id)
            input_tensor = torch.cat([
                input_tensor,
                torch.tensor([[next_token_id]], dtype=torch.long).to(device)
            ], dim=1)
    
    return tokenizer.decode(generated_ids)

# Test nucleus sampling with different parameters
prompt = "ATGCGATCGAT"
print(f"Prompt: {prompt}\n")

for p_val in [0.5, 0.7, 0.9]:
    for temp in [0.8, 1.0, 1.2]:
        generated = nucleus_generate(
            model, tokenizer, prompt, 
            max_length=30, p=p_val, temperature=temp, device=device
        )
        print(f"p={p_val}, temp={temp}: {generated}")

### 3. Top-k Sampling

In [ ]:
def top_k_sample(logits: torch.Tensor, k: int = 50) -> int:
    """
    Sample from top-k distribution.
    """
    # Get top-k logits and indices
    top_k_logits, top_k_indices = torch.topk(logits, k)
    
    # Sample from top-k distribution
    probs = F.softmax(top_k_logits, dim=-1)
    next_token_idx = torch.multinomial(probs, num_samples=1)
    next_token = top_k_indices[next_token_idx]
    
    return next_token.item()

def top_k_generate(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    prompt: str,
    max_length: int = 100,
    k: int = 50,
    temperature: float = 1.0,
    device: torch.device = torch.device('cpu')
) -> str:
    """
    Generate text using top-k sampling.
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt)
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    
    generated_ids = input_ids.copy()
    
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(input_tensor)
            logits = outputs['logits'][0, -1, :] / temperature
            
            next_token_id = top_k_sample(logits, k)
            
            if next_token_id == tokenizer.eos_token_id:
                break
                
            generated_ids.append(next_token_id)
            input_tensor = torch.cat([
                input_tensor,
                torch.tensor([[next_token_id]], dtype=torch.long).to(device)
            ], dim=1)
    
    return tokenizer.decode(generated_ids)

# Test top-k sampling
prompt = "ATGCGATCGAT"
print(f"Prompt: {prompt}\n")

for k_val in [10, 25, 50]:
    generated = top_k_generate(
        model, tokenizer, prompt, 
        max_length=30, k=k_val, temperature=1.0, device=device
    )
    print(f"k={k_val}: {generated}")

## Conditional Generation

### 1. Sequence Completion

In [ ]:
def complete_sequence(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    partial_sequence: str,
    target_length: int,
    method: str = 'nucleus',
    **kwargs
) -> str:
    """
    Complete a partial genomic sequence to target length.
    """
    current_length = len(partial_sequence)
    remaining_length = target_length - current_length
    
    if remaining_length <= 0:
        return partial_sequence[:target_length]
    
    generation_methods = {
        'greedy': greedy_generate,
        'nucleus': nucleus_generate,
        'top_k': top_k_generate
    }
    
    generate_fn = generation_methods.get(method, nucleus_generate)
    
    completed = generate_fn(
        model, tokenizer, partial_sequence,
        max_length=remaining_length,
        device=device,
        **kwargs
    )
    
    return completed[:target_length]

# Test sequence completion
partial_sequences = [
    "ATGCGATCG",
    "AAATTTGGGCCC",
    "CGTACGTACGTA"
]

target_length = 50

print("Sequence Completion Examples:\n")
for seq in partial_sequences:
    completed = complete_sequence(
        model, tokenizer, seq, target_length,
        method='nucleus', p=0.8, temperature=0.9
    )
    print(f"Partial:   {seq}")
    print(f"Completed: {completed}")
    print(f"Length:    {len(completed)}\n")

### 2. Motif-Conditioned Generation

In [ ]:
def generate_with_motif(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    motif: str,
    context_length: int = 100,
    method: str = 'nucleus',
    **kwargs
) -> str:
    """
    Generate sequence containing a specific motif.
    """
    # Generate prefix
    prefix_length = context_length // 2
    prefix_prompt = "N" * 10  # Generic start
    
    prefix = nucleus_generate(
        model, tokenizer, prefix_prompt,
        max_length=prefix_length, device=device, **kwargs
    )
    
    # Insert motif
    sequence_with_motif = prefix + motif
    
    # Generate suffix
    suffix_length = context_length - len(sequence_with_motif)
    if suffix_length > 0:
        complete_sequence_result = nucleus_generate(
            model, tokenizer, sequence_with_motif,
            max_length=suffix_length, device=device, **kwargs
        )
        return complete_sequence_result
    
    return sequence_with_motif

# Test motif-conditioned generation
motifs = ["TATAAA", "CCAAT", "GGGCGG"]

print("Motif-Conditioned Generation:\n")
for motif in motifs:
    generated = generate_with_motif(
        model, tokenizer, motif,
        context_length=80, p=0.8, temperature=0.9
    )
    print(f"Motif: {motif}")
    print(f"Generated: {generated}")
    print(f"Motif present: {motif in generated}\n")

## Generation Quality Assessment

In [ ]:
def assess_generation_quality(
    generated_sequences: List[str],
    reference_sequences: Optional[List[str]] = None
) -> Dict[str, float]:
    """
    Assess quality of generated sequences.
    """
    metrics = {}
    
    # Basic sequence statistics
    lengths = [len(seq) for seq in generated_sequences]
    metrics['avg_length'] = np.mean(lengths)
    metrics['std_length'] = np.std(lengths)
    
    # Nucleotide composition
    all_nucleotides = ''.join(generated_sequences)
    total_chars = len(all_nucleotides)
    
    if total_chars > 0:
        metrics['gc_content'] = (all_nucleotides.count('G') + all_nucleotides.count('C')) / total_chars
        metrics['at_content'] = (all_nucleotides.count('A') + all_nucleotides.count('T')) / total_chars
    
    # Complexity (entropy)
    entropies = []
    for seq in generated_sequences:
        if len(seq) > 0:
            char_counts = {}
            for char in seq:
                char_counts[char] = char_counts.get(char, 0) + 1
            
            probs = np.array(list(char_counts.values())) / len(seq)
            entropy = -np.sum(probs * np.log2(probs + 1e-10))
            entropies.append(entropy)
    
    metrics['avg_entropy'] = np.mean(entropies) if entropies else 0
    
    # Repetitiveness (simple metric)
    repetition_scores = []
    for seq in generated_sequences:
        if len(seq) >= 4:
            # Count 4-mer repeats
            kmers = {}
            for i in range(len(seq) - 3):
                kmer = seq[i:i+4]
                kmers[kmer] = kmers.get(kmer, 0) + 1
            
            total_kmers = len(seq) - 3
            unique_kmers = len(kmers)
            repetition = 1 - (unique_kmers / total_kmers) if total_kmers > 0 else 0
            repetition_scores.append(repetition)
    
    metrics['avg_repetition'] = np.mean(repetition_scores) if repetition_scores else 0
    
    return metrics

# Generate multiple sequences for quality assessment
prompt = "ATGCGATCGAT"
num_sequences = 10
generated_sequences = []

print("Generating sequences for quality assessment...")
for i in tqdm(range(num_sequences)):
    seq = nucleus_generate(
        model, tokenizer, prompt,
        max_length=100, p=0.8, temperature=1.0, device=device
    )
    generated_sequences.append(seq)

# Assess quality
quality_metrics = assess_generation_quality(generated_sequences)

print("\nGeneration Quality Metrics:")
for metric, value in quality_metrics.items():
    print(f"{metric}: {value:.4f}")

## Visualization of Generation Patterns

In [ ]:
def plot_generation_analysis(generated_sequences: List[str]):
    """
    Create visualizations for generation analysis.
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Length distribution
    lengths = [len(seq) for seq in generated_sequences]
    axes[0, 0].hist(lengths, bins=20, alpha=0.7, color='skyblue')
    axes[0, 0].set_title('Sequence Length Distribution')
    axes[0, 0].set_xlabel('Length')
    axes[0, 0].set_ylabel('Count')
    
    # 2. Nucleotide composition
    nucleotides = ['A', 'T', 'G', 'C']
    compositions = []
    
    for nt in nucleotides:
        total_count = sum(seq.count(nt) for seq in generated_sequences)
        total_length = sum(len(seq) for seq in generated_sequences)
        compositions.append(total_count / total_length if total_length > 0 else 0)
    
    axes[0, 1].bar(nucleotides, compositions, color=['red', 'blue', 'green', 'orange'])
    axes[0, 1].set_title('Nucleotide Composition')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. GC content distribution
    gc_contents = []
    for seq in generated_sequences:
        if len(seq) > 0:
            gc_count = seq.count('G') + seq.count('C')
            gc_content = gc_count / len(seq)
            gc_contents.append(gc_content)
    
    axes[1, 0].hist(gc_contents, bins=15, alpha=0.7, color='lightgreen')
    axes[1, 0].set_title('GC Content Distribution')
    axes[1, 0].set_xlabel('GC Content')
    axes[1, 0].set_ylabel('Count')
    
    # 4. Sequence complexity (entropy)
    entropies = []
    for seq in generated_sequences:
        if len(seq) > 0:
            char_counts = {}
            for char in seq:
                char_counts[char] = char_counts.get(char, 0) + 1
            
            probs = np.array(list(char_counts.values())) / len(seq)
            entropy = -np.sum(probs * np.log2(probs + 1e-10))
            entropies.append(entropy)
    
    axes[1, 1].hist(entropies, bins=15, alpha=0.7, color='coral')
    axes[1, 1].set_title('Sequence Complexity (Entropy)')
    axes[1, 1].set_xlabel('Entropy (bits)')
    axes[1, 1].set_ylabel('Count')
    
    plt.tight_layout()
    plt.show()

# Create visualizations
plot_generation_analysis(generated_sequences)

## Advanced Generation Techniques

### 1. Beam Search

In [ ]:
def beam_search_generate(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    prompt: str,
    max_length: int = 50,
    beam_size: int = 5,
    device: torch.device = torch.device('cpu')
) -> List[Tuple[str, float]]:
    """
    Generate text using beam search.
    Returns list of (sequence, score) tuples.
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt)
    
    # Initialize beams: (sequence, score)
    beams = [(input_ids, 0.0)]
    completed_beams = []
    
    with torch.no_grad():
        for step in range(max_length):
            candidates = []
            
            for sequence, score in beams:
                if len(sequence) >= max_length or sequence[-1] == tokenizer.eos_token_id:
                    completed_beams.append((sequence, score))
                    continue
                
                # Get model predictions
                input_tensor = torch.tensor([sequence], dtype=torch.long).to(device)
                outputs = model(input_tensor)
                logits = outputs['logits'][0, -1, :]
                log_probs = F.log_softmax(logits, dim=-1)
                
                # Get top-k candidates
                top_log_probs, top_indices = torch.topk(log_probs, beam_size)
                
                for i in range(beam_size):
                    new_sequence = sequence + [top_indices[i].item()]
                    new_score = score + top_log_probs[i].item()
                    candidates.append((new_sequence, new_score))
            
            # Select top beam_size candidates
            candidates.sort(key=lambda x: x[1], reverse=True)
            beams = candidates[:beam_size]
            
            if not beams:
                break
    
    # Add remaining beams to completed
    completed_beams.extend(beams)
    
    # Sort by score and decode
    completed_beams.sort(key=lambda x: x[1], reverse=True)
    
    results = []
    for sequence, score in completed_beams[:beam_size]:
        decoded = tokenizer.decode(sequence)
        results.append((decoded, score))
    
    return results

# Test beam search
prompt = "ATGCGATCGAT"
beam_results = beam_search_generate(
    model, tokenizer, prompt,
    max_length=30, beam_size=5, device=device
)

print(f"Beam Search Results for prompt: {prompt}\n")
for i, (sequence, score) in enumerate(beam_results):
    print(f"Beam {i+1} (score: {score:.3f}): {sequence}")

### 2. Constrained Generation

In [ ]:
def constrained_generate(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    prompt: str,
    constraints: Dict[str, any],
    max_length: int = 100,
    device: torch.device = torch.device('cpu')
) -> str:
    """
    Generate sequence with constraints.
    
    Constraints can include:
    - 'forbidden_patterns': List of patterns to avoid
    - 'required_patterns': List of patterns that must be included
    - 'gc_content_range': (min, max) GC content
    - 'max_repeats': Maximum consecutive repeats of same nucleotide
    """
    model.eval()
    
    input_ids = tokenizer.encode(prompt)
    input_tensor = torch.tensor([input_ids], dtype=torch.long).to(device)
    
    generated_ids = input_ids.copy()
    
    forbidden_patterns = constraints.get('forbidden_patterns', [])
    required_patterns = constraints.get('required_patterns', [])
    gc_range = constraints.get('gc_content_range', (0.0, 1.0))
    max_repeats = constraints.get('max_repeats', float('inf'))
    
    with torch.no_grad():
        for _ in range(max_length):
            outputs = model(input_tensor)
            logits = outputs['logits'][0, -1, :]
            
            # Apply constraints by modifying logits
            for token_id in range(len(logits)):
                token = tokenizer.decode([token_id])
                candidate_sequence = tokenizer.decode(generated_ids + [token_id])
                
                # Check forbidden patterns
                valid = True
                for pattern in forbidden_patterns:
                    if pattern in candidate_sequence:
                        logits[token_id] = float('-inf')
                        valid = False
                        break
                
                if not valid:
                    continue
                
                # Check repeat constraint
                if len(generated_ids) > 0:
                    last_token = tokenizer.decode([generated_ids[-1]])
                    if token == last_token:
                        # Count consecutive repeats
                        repeat_count = 1
                        for i in range(len(generated_ids) - 2, -1, -1):
                            if tokenizer.decode([generated_ids[i]]) == token:
                                repeat_count += 1
                            else:
                                break
                        
                        if repeat_count >= max_repeats:
                            logits[token_id] = float('-inf')
                            continue
            
            # Sample from constrained distribution
            probs = F.softmax(logits, dim=-1)
            next_token_id = torch.multinomial(probs, num_samples=1).item()
            
            if next_token_id == tokenizer.eos_token_id:
                break
            
            generated_ids.append(next_token_id)
            input_tensor = torch.cat([
                input_tensor,
                torch.tensor([[next_token_id]], dtype=torch.long).to(device)
            ], dim=1)
    
    return tokenizer.decode(generated_ids)

# Test constrained generation
prompt = "ATGCGATCGAT"
constraints = {
    'forbidden_patterns': ['AAAA', 'TTTT'],
    'max_repeats': 3
}

constrained_seq = constrained_generate(
    model, tokenizer, prompt, constraints,
    max_length=50, device=device
)

print(f"Prompt: {prompt}")
print(f"Constraints: {constraints}")
print(f"Generated: {constrained_seq}")

# Verify constraints
print("\nConstraint Verification:")
print(f"Contains AAAA: {'AAAA' in constrained_seq}")
print(f"Contains TTTT: {'TTTT' in constrained_seq}")

## Generation Evaluation and Comparison

In [ ]:
def compare_generation_methods(
    model: torch.nn.Module,
    tokenizer: GenomicTokenizer,
    prompts: List[str],
    methods: List[str],
    device: torch.device = torch.device('cpu')
) -> Dict[str, Dict[str, any]]:
    """
    Compare different generation methods.
    """
    results = {}
    
    for method in methods:
        method_results = []
        
        for prompt in prompts:
            if method == 'greedy':
                generated = greedy_generate(
                    model, tokenizer, prompt, max_length=50, device=device
                )
            elif method == 'nucleus':
                generated = nucleus_generate(
                    model, tokenizer, prompt, max_length=50,
                    p=0.8, temperature=1.0, device=device
                )
            elif method == 'top_k':
                generated = top_k_generate(
                    model, tokenizer, prompt, max_length=50,
                    k=50, temperature=1.0, device=device
                )
            
            method_results.append(generated)
        
        # Compute metrics for this method
        metrics = assess_generation_quality(method_results)
        results[method] = {
            'sequences': method_results,
            'metrics': metrics
        }
    
    return results

# Compare generation methods
test_prompts = [
    "ATGCGATCGAT",
    "AAATTTGGGCCC",
    "CGTACGTACGTA",
    "TATATATATAT",
    "GCGCGCGCGCG"
]

methods_to_compare = ['greedy', 'nucleus', 'top_k']

print("Comparing generation methods...")
comparison_results = compare_generation_methods(
    model, tokenizer, test_prompts, methods_to_compare, device
)

# Display comparison
print("\nGeneration Method Comparison:")
print("=" * 50)

for method, data in comparison_results.items():
    print(f"\n{method.upper()} METHOD:")
    print("-" * 20)
    
    metrics = data['metrics']
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")
    
    print("\nSample generations:")
    for i, seq in enumerate(data['sequences'][:3]):
        print(f"  {i+1}. {seq[:60]}..." if len(seq) > 60 else f"  {i+1}. {seq}")

## Interactive Generation Demo

In [ ]:
def interactive_generation_demo():
    """
    Interactive demo for trying different generation parameters.
    """
    print("Interactive Generation Demo")
    print("=" * 40)
    
    # Default parameters
    params = {
        'prompt': 'ATGCGATCGAT',
        'method': 'nucleus',
        'max_length': 50,
        'temperature': 1.0,
        'p': 0.8,
        'k': 50
    }
    
    print(f"\nCurrent parameters:")
    for key, value in params.items():
        print(f"  {key}: {value}")
    
    # Generate with current parameters
    if params['method'] == 'nucleus':
        generated = nucleus_generate(
            model, tokenizer, params['prompt'],
            max_length=params['max_length'],
            p=params['p'],
            temperature=params['temperature'],
            device=device
        )
    elif params['method'] == 'top_k':
        generated = top_k_generate(
            model, tokenizer, params['prompt'],
            max_length=params['max_length'],
            k=params['k'],
            temperature=params['temperature'],
            device=device
        )
    else:  # greedy
        generated = greedy_generate(
            model, tokenizer, params['prompt'],
            max_length=params['max_length'],
            device=device
        )
    
    print(f"\nGenerated sequence:")
    print(f"  {generated}")
    
    # Quality assessment
    quality = assess_generation_quality([generated])
    print(f"\nQuality metrics:")
    for metric, value in quality.items():
        print(f"  {metric}: {value:.4f}")

# Run interactive demo
interactive_generation_demo()

## Summary and Best Practices

### Key Takeaways

1. **Method Selection**: Choose generation method based on requirements:
   - **Greedy**: Fast, deterministic, but may be repetitive
   - **Nucleus (Top-p)**: Good balance of quality and diversity
   - **Top-k**: Controllable diversity
   - **Beam Search**: Higher quality but computationally expensive

2. **Parameter Tuning**:
   - Lower temperature (0.6-0.8) for more focused generation
   - Higher temperature (1.0-1.2) for more diverse generation
   - p=0.8-0.9 for nucleus sampling
   - k=25-50 for top-k sampling

3. **Quality Assessment**:
   - Monitor GC content for biological realism
   - Check sequence complexity (entropy)
   - Avoid excessive repetition
   - Validate against known biological constraints

4. **Constraints and Control**:
   - Use constrained generation for specific requirements
   - Implement post-processing filters
   - Consider domain-specific constraints

5. **Evaluation**:
   - Compare multiple generation methods
   - Use diverse evaluation metrics
   - Consider downstream task performance

In [ ]:
print("Generation Tutorial Complete!")
print("\nThis notebook covered:")
print("- Greedy, nucleus, and top-k sampling")
print("- Beam search generation")
print("- Constrained generation")
print("- Quality assessment and comparison")
print("- Interactive generation demo")
print("\nFor more advanced techniques, see the Hyena-GLT documentation.")